In [44]:
import cv2
import pandas as pd
import os
from matplotlib import pyplot as plt

In [32]:
df = []
for file in os.listdir('ocr/csv'):
    df.append(pd.read_csv(f'ocr/csv/{file}'))

df = pd.concat(df, axis=0)

In [33]:
df = df[~df['label'].isnull()].reset_index(drop=True)
print(df.shape)

(1997, 2)


In [34]:
path_image = '/home/dmk/Documents/LicenceOCR/darknet/data/images'
path_label = '/home/dmk/Documents/LicenceOCR/darknet/data/labels'
list_file_label = os.listdir(path_label)

In [53]:
def split_df(df, ratio: float = 0.9):
    n_train = int(len(df) * ratio)
    return df[:n_train], df[n_train:]
    

def read_coordinate(file_label):
    with open(f"{path_label}/{file_label}.txt", 'r') as file:
        data = file.readlines()
    coordinate = data[0].strip().split(' ')[1:5]
    return [float(value) for value in coordinate]
    
def crop_image(file_image):
    image = cv2.imread(f"{path_image}/{file_image}")
    file_name, extension = os.path.splitext(file_image)
    height, width, _ = image.shape
    coordinate = read_coordinate(file_name)
    x, y, w, h = int(width * coordinate[0]), int(height  * coordinate[1]), int(width * coordinate[2]), int(height * coordinate[3])
    image = image[int(y - h/2): int(y + h/2):, int(x - w/2): int(x + w/2), :]
    cv2.imwrite(f'ocr/images/{file_image}', image)


In [49]:
df['is_car'] = df['file_name'].apply(lambda x: 'car' in x)

In [50]:
df_moto = df[df['is_car'] == False].reset_index()
df_car = df[df['is_car'] == True].reset_index()
df_moto_train, df_moto_test = split_df(df_moto)
df_car_train, df_car_test = split_df(df_car)
df_train = pd.concat((df_moto_train, df_car_train), axis=0)
df_val = pd.concat((df_moto_test, df_car_test))

In [54]:
def save_file(df, mode: str = 'train'):
    with open(f'ocr/{mode}.txt', 'w') as file:
        for idx, row in df.iterrows():
            image = row['file_name']
            label = row['label']
            crop_image(image)
            file.write(f"ocr/images/{image}" + '\t' + label + '\n')

In [55]:
save_file(df_train)

In [57]:
save_file(df_val, mode='val')

In [45]:
image = cv2.imread('darknet/data/images/car_AEONTP_6A00530_checkin_2020-1-13-16-16i7h_Z3Ua0Y.jpg')
height, width, _ = image.shape
x, y, w, h = int(width * 0.503000), int(height * 0.653291), int(width * 0.646000), int(height*0.115570)
cv2.imwrite('temp.png', image[int(y - h/2): int(y + h/2),int(x - w/2): int(x + w/2) ])

True